In [2]:
import pandas as pd
import pickle
import os
from os.path import join
import re
import tqdm
from file_function import re_name_upload, write_log_csv_upload
from tqdm import tqdm

##### 채무자조회.pkl 만들기

In [3]:
# 현재 버전 221118
path = r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일"
excel_file = "채무자조회.xlsx"
df = pd.read_excel(join(path, excel_file))
len(df)

70792

In [4]:
pkl_file = "채무자조회.pkl"
df.to_pickle(join(path, pkl_file))

##### 신규 종결건 처리하기

In [5]:
df_old = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\채무자조회 구버전\채무자조회_221021.pkl").fillna("")
df_new = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\채무자조회.pkl").fillna("")

In [6]:
df1 = df_old.loc[:, ["채무자키", "채무상태"]]
df2 = df_new.loc[:, ["채무자키", "채무상태", "매각사구분"]]

In [9]:
df3 = pd.merge(df1,df2, on="채무자키", how="outer")
df3.columns = ["채무자키", "old", "new", "매각사구분"]
df3

,채무자키,old,new,매각사구분
0,20495151,정상,정상,전북-01
1,20495152,정상,정상,전북-01
2,20495238,정상,정상,전북-01
3,20417854,정상,정상,DNP-01
4,20411449,정상,정상,DNP-01
...,...,...,...,...
70787,20525075,NaN,신용회복(확정),카카오-01
70788,20526239,NaN,신용회복(확정),카카오-01
70789,20527282,NaN,신용회복(확정),카카오-01
70790,20525073,NaN,개인회생(확정),카카오-01


In [17]:
df4 = df3[~(df3["old"].str.match(r"개인회생\(면책\)|파산\(면책\)|환매|매각|종결", na=False)) & (df3["new"].str.match(r"개인회생\(면책\)|파산\(면책\)|환매|매각|종결", na=False))]
df4

,채무자키,old,new,매각사구분
67,20495134,정상,종결,전북-01
480,20432765,정상,파산(면책),KB카드-01
579,20433694,정상,파산(면책),KB카드-01
941,20502810,정상,종결,스마트저축-01
1397,20421481,정상,종결,한울가람
...,...,...,...,...
67263,20441028,정상,파산(면책),KB카드-01
67499,20431563,정상,종결,KB카드-01
67668,20408935,개인회생(확정),개인회생(면책),대산
70116,20527419,NaN,종결,카카오-01


In [21]:
#보고용
# df5 = df3[df3["new"].str.match(r"개인회생\(면책\)|파산\(면책\)|환매|매각|종결")]["채무자키"]
# df5.index = range(len(df5))
# df5.to_excel("c://Users/SL/Desktop/관리제외221021.xlsx")

In [22]:
# 신규 종결건 보고용
# df4.columns = ["채무자키", "채무상태220906", "채무상태221021", "매각사구분"]
# df4.reset_index(inplace=True, drop=True)
# df4.to_excel("c://Users/SL/Desktop/신규종결건.xlsx")

In [18]:
result = [(str(x), y) for x, y in zip(df4.채무자키, df4.매각사구분)]
len(result)

296

In [19]:
docu_folder_dict = {"원인서류": "1.원인서류", "양도통지서": "2.양도통지서", "집행권원": "3.집행권원", "강제집행": "4.강제집행", 
        "등본": "5.등초본", "법인등기" : "5.등초본", "초본": "5.등초본", "등초본":"5.등초본", "외국인증명": "5.등초본", "주민등록정보":"5.등초본", 
        "개인회생": "6.개인회생", "신용회복": "7.신용회복", "파산": "8.파산", "재산조사": "9.재산조사", "부채증명서" : "10.부채증명서", 
        "신용조회" : "11.신용조회", "개인정보":"12.개인정보", "기타": "기타"}
depth1 = list(docu_folder_dict.values())
depth1

['1.원인서류',
 '2.양도통지서',
 '3.집행권원',
 '4.강제집행',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '6.개인회생',
 '7.신용회복',
 '8.파산',
 '9.재산조사',
 '10.부채증명서',
 '11.신용조회',
 '12.개인정보',
 '기타']

###### 신규 종결건 파일서버에서 삭제하기

In [20]:
# path에 신규 종결건 파일이 있는 경우만 골라서 관리제외 폴더로 옮기기
path = r"\\192.168.0.75\솔림헬프"
out_dir_base = r"\\192.168.0.75\삭제예정파일\관리제외"
path_log = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전'
success = []
fail = []
dir_cnt = 0

for d1 in tqdm(depth1) : 
    for d3, d2 in result :
        src_d = join(path, d1, d2, d3)
        if os.path.exists(src_d) :

            dst_d = join(out_dir_base, d2, d3)
            if not os.path.exists(dst_d) :
                os.makedirs(dst_d)

            for f in os.listdir(src_d) :  # 파일 옮기기
                try :
                    success.append(re_name_upload(join(src_d, f), join(dst_d, f)))
                except Exception as e :    
                        fail.append([src_d, f, dst_d, f, e.__class__])
                        continue
            
            try : 
                os.rmdir(src_d) # 빈폴더 삭제하기
                dir_cnt += 1
            except Exception as e : 
                print(e)
            
print(len(success),"개의 파일 이동")
print(len(fail),"개의 파일 이동 실패")
print(dir_cnt,"개의 폴더 삭제")
write_log_csv_upload(success, path_log, "success")
write_log_csv_upload(fail, path_log, "fail")

100%|██████████| 18/18 [21:30<00:00, 71.68s/it]  


1308 개의 파일 이동
0 개의 파일 이동 실패
799 개의 폴더 삭제
log 파일이 비어있습니다. path : \\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전


In [23]:
dir_cnt

799

In [109]:
# 전체 os.walk 하는 비효율적인 코드
# path = r"\\192.168.0.75\솔림헬프"
# out_dir_base = r"\\192.168.0.75\관리제외"
# path_log = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전'
# success = []
# fail = []
# dir_cnt = 0
# for root, __dirs__, files in tqdm(os.walk(path)) :
#     if re.search("#recycle", root) == None : # 휴지통 nono
#         for key in result :
#             try : 
#                 if re.search(key, root) :
#                     depth2 = root.split(os.path.sep)[-2]
#                     depth3 = root.split(os.path.sep)[-3]
#                     out_dir = join(out_dir_base, depth2, depth3)
                    
#                     # if not os.path.exists(out_dir) :
#                     #     os.makedirs(out_dir)
#                     try :
#                         for f in os.listdir(root) :
#                             success.append([join(root, f), join(out_dir, f)])
#                             # success.append(re_name_upload(join(root, f), join(out_dir, f)))
#                     except Exception as e :    
#                         fail.append([root, f, out_dir, f, e.__class__])
#                         continue
            
#                     # os.rmdir(out_dir)
#                     # dir_cnt += 1
#                     break
#             except Exception as e : 
#                 print(e.with_traceback())
#                 continue
                
# print(len(success),"개의 파일 이동")
# print(len(fail),"개의 파일 이동 실패")
# print(dir_cnt,"개의 폴더 삭제")
# write_log_csv_upload(success, path_log, "success")
# write_log_csv_upload(fail, path_log, "fail")

453 개의 파일 이동
0 개의 파일 이동 실패
0 개의 폴더 삭제
log 파일이 비어있습니다. path : \\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전


##### 채무자조회 컬럼 추가

In [14]:
import pandas as pd
import pickle

In [16]:
with open(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\계좌조회새창_20221124_1008.pkl", "rb") as pkl :
    df_left = pickle.load(pkl)
df_left

,채무자키,매각사구분,계좌번호,최초대출일,최초연체일,은행대출원금,최초원금,원채권사,원채권키,채무자명,채무자주민번호,채무상태,타채무자키,타계좌키
0,20412714,DNP-02,1375-16-01001756,2003-04-30,NaN,400000000,400000000,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
1,20412714,DNP-02,5052-16-10081511,2002-06-27,NaN,300000000,250000000,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
2,20412714,DNP-02,5052-16-10081863,2002-07-23,NaN,200000000,197799852,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
3,20412714,DNP-02,5052-16-10082013,2002-07-30,NaN,100000000,100000000,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
4,20455180,DNP-01,8288-09-01017811,2000-10-20,NaN,0,0,국민은행,0,전윤선,541012-2683023,매각,0405001221,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78149,20527985,카카오-01,3650031435424,2018-04-06,2022-04-07,37500000,36765870,카카오뱅크,0,김종연,750529-1390310,신용회복(확정),3650031435424,CCRS-467
78150,20527986,카카오-01,3650011051361,2017-10-09,2019-10-11,5000000,3251178,카카오뱅크,0,정윤미,841003-2652623,신용회복(확정),3650011051361,CCRS-1023
78151,20527987,카카오-01,3650013698082,2018-09-21,2021-06-18,21000000,21000000,카카오뱅크,0,박정업,720215-1221241,개인회생(확정),3650013698082,IRL-846
78152,20527988,카카오-01,3650030453818,2019-06-12,2020-12-10,3000000,22777210,카카오뱅크,0,김태수,890406-1111416,개인회생(확정),3650030453818,IRL-815


In [17]:
# df_right = pd.read_excel(r'c://Users/SL/Desktop/')
# df_right
with open(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\전화 주소_20221124_1313.pkl", "rb") as pkl :
    df_right = pickle.load(pkl)
df_right

,채무자키,자택전화인,직장전화인,핸드폰인,자택주소,자택번지인,직장주소,직장번지인,초본주소,초본번지인
0,20495151,063-227-0416,NaN,NaN,전라북도 임실군 신평면,석등슬치로 354-1,NaN,NaN,NaN,NaN
1,20495152,063-227-0416,NaN,NaN,전라북도 임실군 신평면,석등슬치로 354-1,NaN,NaN,NaN,NaN
2,20495238,063-227-0416,NaN,NaN,전라북도 임실군 신평면,석등슬치로 354-1,NaN,NaN,NaN,NaN
3,20417854,063-832-5411,063-834-9912,011-944-6480,전북 익산시 어양동,510-1 어양아파트 12-411,전북 익산시 어양동,657-4번지,NaN,NaN
4,20411449,NaN,063-644-3008,010-4652-6956,NaN,NaN,전북 임실군 오수면,금암리 761-3번지,전북 임실군 오수면 금암1길 39-16,(금암리)
...,...,...,...,...,...,...,...,...,...,...
70787,20459682,NaN,051-660-3056,010-9512-0243,부산 남구 용소로64번길 60,"301호 (대연동,쿨하우스)",부산 남구 용당동,산100번지 부경대학교용당캠퍼스 한미르관 4층,NaN,NaN
70788,20459636,NaN,051-205-5177,010-8687-5147,부산 강서구 녹산산업중로 409,(송정동),부산 강서구 녹산산업중로 409,(송정동),NaN,NaN
70789,20447783,NaN,043-530-3200,NaN,서울 용산구 이태원로 223,534호 (한남동),충북 진천군 이월면 생거진천로 1770,(중산리),"전북 익산시 고봉로 146,","7동306호 (마동,주공1차아파트)"
70790,20459338,NaN,NaN,010-7659-9939,서울 용산구 신흥로11나길 17,(용산동2가),0,0,NaN,NaN


In [5]:
# 삭제할 col 있는 경우
# df_right.drop(['은행대출원금', '최초원금'], axis=1, inplace=True)
# df_right

,채무자키,현재원금
0,20455180,0
1,20455180,0
2,20455181,0
3,20411129,0
4,20412715,0
...,...,...
78149,20419638,999109587
78150,20429686,0
78151,20430085,0
78152,20494438,89420225


In [18]:
# 결과가 이상하면 기준셀이 key가 아닌지 확인하라.
df_result = pd.merge(df_left, df_right, how='left')
df_result

,채무자키,매각사구분,계좌번호,최초대출일,최초연체일,은행대출원금,최초원금,원채권사,원채권키,채무자명,...,타계좌키,자택전화인,직장전화인,핸드폰인,자택주소,자택번지인,직장주소,직장번지인,초본주소,초본번지인
0,20412714,DNP-02,1375-16-01001756,2003-04-30,NaN,400000000,400000000,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
1,20412714,DNP-02,5052-16-10081511,2002-06-27,NaN,300000000,250000000,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
2,20412714,DNP-02,5052-16-10081863,2002-07-23,NaN,200000000,197799852,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
3,20412714,DNP-02,5052-16-10082013,2002-07-30,NaN,100000000,100000000,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
4,20455180,DNP-01,8288-09-01017811,2000-10-20,NaN,0,0,국민은행,0,전윤선,...,NaN,NaN,NaN,NaN,경북 경산시 삼북동,266-37,NaN,NaN,"경북 경주시 건천읍 부산성길 271-28,",(만교사)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78149,20527985,카카오-01,3650031435424,2018-04-06,2022-04-07,37500000,36765870,카카오뱅크,0,김종연,...,CCRS-467,NaN,NaN,010-9138-4420,서울 양천구 목동중앙본로,10 (극동늘푸른아파트) 101-604,NaN,NaN,NaN,NaN
78150,20527986,카카오-01,3650011051361,2017-10-09,2019-10-11,5000000,3251178,카카오뱅크,0,정윤미,...,CCRS-1023,NaN,NaN,010-4748-0609,서울특별시 동대문구 장한로,"111-1, 903호 (장안동, sm타워)",NaN,NaN,NaN,NaN
78151,20527987,카카오-01,3650013698082,2018-09-21,2021-06-18,21000000,21000000,카카오뱅크,0,박정업,...,IRL-846,NaN,NaN,010-6235-2501,충청남도 아산시 음봉면,"음봉로 847, 101-415 (삼일원앙아파트)",NaN,NaN,NaN,NaN
78152,20527988,카카오-01,3650030453818,2019-06-12,2020-12-10,3000000,22777210,카카오뱅크,0,김태수,...,IRL-815,NaN,NaN,010-5125-9927,부산 사상구 광장로86번길,32 (E클레스) 101호,NaN,NaN,NaN,NaN


In [19]:
df_result.to_pickle(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\계좌_주소_통합본.pkl")

##### df(pkl) to dict(pkl)

In [18]:
path_df = r'C:\Users\SL\Desktop\workspace\python\fileNaming\파일/채무자조회.pkl' 
def dict_referFnc(path_df):
    """dict_refer["key"][0:매각사, 1:채무상태, 2:채무자성명, 3:보증인성명]"""
    df_c = pd.read_pickle(path_df).fillna("")
    #############################
    return dict(map(lambda x : (str(x[1].채무자키),[x[1].매각사구분, x[1].채무상태, x[1].성명, x[1].보증인성명]), df_c.iterrows()))
    #############################
dict_refer = dict_referFnc(path_df) ################## 전역변수로 둬야 함. 함수에서도 쓰고, main에서도 쓴다.
dict_refer

{'20495151': ['전북-01', '정상', '(유)가람건설', '김원목'],
 '20495152': ['전북-01', '정상', '(유)가람건설', '김원목'],
 '20495238': ['전북-01', '정상', '(유)가람건설', '김원목'],
 '20417854': ['DNP-01', '정상', '(유)가람관광여행사', '김길배'],
 '20411449': ['DNP-01', '정상', '(유)거성산업', '남옥례,형공우'],
 '20495020': ['전북-01', '정상', '(유)거시기건설', '정영진'],
 '20495027': ['전북-01', '정상', '(유)거시기건설', '정영진'],
 '20495257': ['전북-01', '정상', '(유)거창인더스', '김도현'],
 '20495258': ['전북-01', '정상', '(유)거창인더스', '김도현'],
 '20495975': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20495981': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496018': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496019': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496035': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496036': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496037': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496038': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496039': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496040': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496041': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496234': ['전북-01', '정상'

In [16]:
path_save = r'C:\Users\SL\Desktop\workspace\python\fileNaming\파일'
fname_save = r'dict_refer.pkl'
with open(join(path_save, fname_save), 'wb') as pkl :
    pickle.dump(dict_refer, pkl)

In [20]:
with open(join(path_save, fname_save), 'rb') as pkl :
    dict_refer1 = pickle.load(pkl)
dict_refer1

{'20495151': ['전북-01', '정상', '(유)가람건설', '김원목'],
 '20495152': ['전북-01', '정상', '(유)가람건설', '김원목'],
 '20495238': ['전북-01', '정상', '(유)가람건설', '김원목'],
 '20417854': ['DNP-01', '정상', '(유)가람관광여행사', '김길배'],
 '20411449': ['DNP-01', '정상', '(유)거성산업', '남옥례,형공우'],
 '20495020': ['전북-01', '정상', '(유)거시기건설', '정영진'],
 '20495027': ['전북-01', '정상', '(유)거시기건설', '정영진'],
 '20495257': ['전북-01', '정상', '(유)거창인더스', '김도현'],
 '20495258': ['전북-01', '정상', '(유)거창인더스', '김도현'],
 '20495975': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20495981': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496018': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496019': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496035': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496036': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496037': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496038': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496039': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496040': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496041': ['전북-01', '정상', '(유)고은조경', '배수산'],
 '20496234': ['전북-01', '정상'

In [21]:
print(len(dict_refer), len(dict_refer1))

70792 70792
